In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding


In [2]:
df = pd.read_csv("processed_reviews.csv")

# Check class distribution
print(df['sentiment'].value_counts())

sentiment
2    9022
1     772
0     467
Name: count, dtype: int64


In [3]:
x = list(df['reviews'])
y = list(df['sentiment'])

In [4]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    x,
    y,
    test_size=0.2,
    random_state=42,
    stratify=df['sentiment']
)

In [5]:
# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize datasets
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.


In [6]:
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)

In [7]:
# Load pre-trained BERT model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    save_strategy="epoch"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer)
)

# Train the model
trainer.train()


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-7-3562514304.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: minendra2062 (minendra2062-self) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,0.709400
100,0.439600
150,0.438200
200,0.354200
250,0.361300
300,0.373000
350,0.320200
400,0.371400
450,0.343500
500,0.311700


TrainOutput(global_step=1539, training_loss=0.26419308651880136, metrics={'train_runtime': 326.1215, 'train_samples_per_second': 75.506, 'train_steps_per_second': 4.719, 'total_flos': 815483848863744.0, 'train_loss': 0.26419308651880136, 'epoch': 3.0})

In [8]:
predictions=trainer.predict(test_dataset)
preds=torch.argmax(torch.tensor(predictions.predictions),axis=1)

print("\nClassification Report: \n")
print(classification_report(test_labels,preds.numpy(),target_names=['Negative','Neutral','Postitive']))


Classification Report: 

              precision    recall  f1-score   support

    Negative       0.67      0.52      0.58        93
     Neutral       0.43      0.41      0.42       155
   Postitive       0.94      0.96      0.95      1805

    accuracy                           0.90      2053
   macro avg       0.68      0.63      0.65      2053
weighted avg       0.89      0.90      0.90      2053



In [10]:
model.eval()

text="This is a bad product."
inputs=tokenizer(text,return_tensors='pt',truncation=True,padding=True,max_length=128)

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs={key: val.to(device) for key, val in inputs.items()}
model.to(device)

outputs=model(**inputs)
logits=outputs.logits
predicted_class_id=torch.argmax(logits,dim=1).item()

print(predicted_class_id)

0
